In [1]:
import pandas as pd
from kmer_model import Bacteria

#df = pd.read_csv('file.csv', encoding='utf-8')

In [2]:
url_gen = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/016/724/805/GCF_016724805.1_ASM1672480v1/GCF_016724805.1_ASM1672480v1_genomic.fna.gz'
url_prot = 'https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Bacteria/UP000005835/UP000005835_742823.fasta.gz'

bacteria = Bacteria(name='Sutterella wadsworthensis', url_prot=url_prot, url_gen=url_gen)

In [3]:
bacteria.get_kmers_gen()

In [4]:
bacteria.get_kmers_prot()

In [ ]:
all_gen, all_prot = {}, {}
corpus_gen, corpus_prot = [], []

for i in range(df.shape[0]):
    bact_name = df['bact'][i]
    bacteria = Bacteria(bact_name, df['gen'][i], df['prot'][i])
    bacteria.get_kmers_gen()
    bacteria.get_kmers_prot()
    
    all_gen[bact_name] = bacteria.gen_dict
    all_prot[bact_name] = bacteria.prot_dict
    
    corpus_gen.append(' '.join(bacteria.kmers_gen['all']))
    corpus_prot.append(' '.join(bacteria.kmers_prot['all']))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=30, max_df=2000)
vectorizer.fit_transform(corpus_gen)

In [ ]:
from gensim.models import FastText

model = FastText(vector_size=50, window=3, min_count=1)
corpus = []

for bact in all_gen:
    kmer_text = []
    for gen in all_gen[bact]:
        kmer_text.append(getKmers(all_gen[bact][gen], 10, 10)) #extend
    corpus.append(kmer_text)
    
model.build_vocab(sentences=corpus)
model.train(sentences=corpus, total_examples=len(corpus), epochs=20)

In [137]:
kmer_len_gen = 30
filename_gen = 'C://Users/User/Downloads/GCF_009721605.1_ASM972160v1_genomic.fna'

gen_dict = {rec.id: str(rec.seq) for rec in SeqIO.parse(filename_gen, 'fasta')}
kmers_gen = {k: list(getKmers(gen_dict[k], kmer_len_gen, 1)) for k in gen_dict}
kmers_gen['all'] = sum(list(kmers_gen.values()), [])
    
kmers_gen_counter = {k: Counter(v) for k,v in kmers_gen.items()}